In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Distributed training with Reduction Server

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/ai-platform-samples/blob/master/ai-platform-unified/notebooks/notebook_template.ipynb"">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/ai-platform-samples/blob/master/ai-platform-unified/notebooks/notebook_template.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

## Overview

This notebook demonstrates how to optimize large distributed training Vertex AI jobs using Reduction Server. 

The machine learning task in this example is fine tuning a BERT model for sentence prediction using  the Multi-Genre Natural Language Inference Corpus (MNLI) from the GLUE benchmark. 

The example uses components from [TensorFlow NLP Modelling Toolkit](https://github.com/tensorflow/models/tree/master/official/nlp#tensorflow-nlp-modelling-toolkit) and distributed training is implemented using [tf.distribute.MultiWorkerMirroredStrategy](https://www.tensorflow.org/api_docs/python/tf/distribute/MultiWorkerMirroredStrategy). 

For more information about using Reduction Server to optimize distributed training refer to the [Optimizing distributed training with Vertex AI Reduction Server](tbd) article.

### Dataset



The example uses the *glue/mnli* dataset from [TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/glue).

### Objective

In this notebook, you will learn how to configure, submit and monitor a Vertex AI custom training job that uses Reduction Server to optimize network bandwith and latency of the gradient reduction operation in the distributed training setting.  

The steps performed include:

- Building a custom training container image based on TensorFlow NLP Modelling Toolkit
- Converting the glue/mnli dataset to the format required by TensorFlow NLP Modelling Toolkit
- Preparing a Vertex AI custom container training job that uses Reduction Server
- Submitting and monitoring the job

### Costs 


This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage


Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Set up your local development environment

**If you are using Colab or Google Cloud Notebooks**, your environment already meets
all the requirements to run this notebook. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK
* Git
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with Python 3

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

1. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

1. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3. Activate the virtual environment.

1. To install Jupyter, run `pip3 install jupyter` on the
command-line in a terminal shell.

1. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

1. Open this notebook in the Jupyter Notebook Dashboard.

### Install the required packages

Install TensorFlow NLP Modelling Toolkit

In [1]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

In [2]:
! pip3 install --upgrade tf-models-official==2.5.0 tensorflow-text==2.5.0

     |████████████████████████████████| 1.6 MB 7.6 MB/s eta 0:00:01
     |████████████████████████████████| 4.3 MB 32.2 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 44.3 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 7.7 MB/s  eta 0:00:01
     |████████████████████████████████| 46 kB 4.6 MB/s  eta 0:00:01
     |████████████████████████████████| 98 kB 6.2 MB/s  eta 0:00:01
     |████████████████████████████████| 99 kB 8.9 MB/s  eta 0:00:01
     |████████████████████████████████| 54 kB 3.3 MB/s  eta 0:00:01
     |████████████████████████████████| 352 kB 50.1 MB/s eta 0:00:01
     |████████████████████████████████| 454.3 MB 7.6 kB/s  eta 0:00:013.9 MB/s eta 0:00:01
     |████████████████████████████████| 38.2 MB 33.2 MB/s eta 0:00:01�██████████████████████▏      | 30.1 MB 33.2 MB/s eta 0:00:01
     |████████████████████████████████| 108 kB 39.6 MB/s eta 0:00:01
     |████████████████████████████████| 211 kB 39.9 MB/s eta 0:00:01
     |███████████████████

Install the latest version of Vertex SDK

In [3]:
! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG

Install the latest GA version of google-cloud-storage library as well 

In [4]:
! pip3 install --upgrade google-cloud-storage $USER_FLAG

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [5]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin


### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API and Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component). {TODO: Update the APIs needed for your tutorial. Edit the API names, and update the link to append the API IDs, separating each one with a comma. For example, container.googleapis.com,cloudbuild.googleapis.com}

1. If you are running this notebook locally, you will need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [1]:
import os

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Project ID:  jk-mlops-dev


Otherwise, set your project ID here.

In [2]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append it onto the name of resources you create in this tutorial.

In [3]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Authenticate your Google Cloud account

**If you are using Google Cloud Notebooks**, your environment is already
authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the Cloud Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. Click **Create service account**.

3. In the **Service account name** field, enter a name, and
   click **Create**.

4. In the **Grant this service account access to project** section, click the **Role** drop-down list. Type "Vertex AI"
into the filter box, and select
   **Vertex AI Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [4]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# If on Google Cloud Notebooks, then don't execute this code
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**


In this example, your training application uses Cloud Storage for accessing training and validation datasets and for storing checkpoints. 

Set the name of your Cloud Storage bucket below. It must be unique across all
Cloud Storage buckets.

You may also change the `REGION` variable, which is used for operations
throughout the rest of this notebook. Make sure to [choose a region where Vertex AI services are
available](https://cloud.google.com/vertex-ai/docs/general/locations#available_regions). You may
not use a Multi-Regional Storage bucket for training with Vertex AI.

In [5]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}
REGION = "[your-region]"  # @param {type:"string"}

In [6]:
BUCKET_NAME = "gs://jk-rs-notebook-test"  # @param {type:"string"}
REGION = "us-central1"  # @param {type:"string"}

In [7]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [8]:
! gsutil mb -l $REGION $BUCKET_NAME

Creating gs://jk-rs-notebook-test/...
ServiceException: 409 A Cloud Storage bucket named 'jk-rs-notebook-test' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


Finally, validate access to your Cloud Storage bucket by examining its contents:

In [9]:
! gsutil ls -al $BUCKET_NAME

### Import libraries and define constants

In [26]:
import json
import pprint
import sys
import shutil
import time
import tensorflow as tf

from official.nlp.bert import tokenization
from official.nlp.data import classifier_data_lib

from google.cloud import aiplatform
from google.cloud.aiplatform_v1beta1 import types
from google.cloud.aiplatform_v1beta1.services.job_service import \
    JobServiceClient

### Vertex constants

Setup up the following constants for Vertex:

- API_ENDPOINT: The Vertex API service endpoint for job services.
- PARENT: The Vertex location root path for job resources.

In [27]:
API_ENDPOINT = '{}-aiplatform.googleapis.com'.format(REGION)
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

## Tutorial


### Set up clients

The Vertex client library works as a client/server model. On your side (the Python script) you will create a client that sends requests and receives responses from the Vertex server.

In this example, you use the Job Service client for submitting and monitoring custom training jobs.

In [28]:
client_options = {"api_endpoint": API_ENDPOINT}
job_client = JobServiceClient(client_options=client_options)

## Prepare training and validation datasets

TensorFlow NLP Modelling Toolkit that you use for fine tuning BERT requires datasets in a specific format. The toolkit includes a set of utility functions to help with data conversions. You will use them to convert the *glue/mnli* dataset from TensorFlow Datasets.

In [34]:
def generate_mnli_tfrecords(train_data_output_path, 
                            eval_data_output_path,
                            metadata_file_path,
                            vocab_file='gs://cloud-tpu-checkpoints/bert/keras_bert/uncased_L-24_H-1024_A-16/vocab.txt', 
                            mnli_type='matched', 
                            max_seq_length=128, 
                            do_lower_case=True):
    """Generates MNLI training and validation splits in the TFRecord format
    compatible with TensorfFlow NLP Modelling Toolkit."""

    tokenizer = tokenization.FullTokenizer(
        vocab_file=vocab_file, do_lower_case=do_lower_case)

    processor_text_fn = tokenization.convert_to_unicode

    if mnli_type == 'matched':
        tfds_params = 'dataset=glue/mnli,text_key=hypothesis,text_b_key=premise,train_split=train,dev_split=validation_matched'
    else: 
        tfds_params = 'dataset=glue/mnli,text_key=hypothesis,text_b_key=premise,train_split=train,dev_split=validation_mismatched'

    processor = classifier_data_lib.TfdsProcessor(
        tfds_params=tfds_params, process_text_fn=processor_text_fn)

    metadata = classifier_data_lib.generate_tf_record_from_data_file(
        processor,
        None,
        tokenizer,
        train_data_output_path=train_data_output_path,
        eval_data_output_path=eval_data_output_path,
        max_seq_length=max_seq_length)

    with tf.io.gfile.GFile(metadata_file_path, "w") as writer:
        writer.write(json.dumps(metadata, indent=4) + "\n")

In [35]:
OUTPUT_LOCATION = f'{BUCKET_NAME}/datasets/MNLI'
TRAIN_FILE = f'{OUTPUT_LOCATION}/mnli_train.tf_record'
EVAL_FILE = f'{OUTPUT_LOCATION}/mnli_valid.tf_record'
METADATA_FILE = f'{OUTPUT_LOCATION}/metadata.json'

In [ ]:
generate_mnli_tfrecords(TRAIN_FILE, EVAL_FILE, METADATA_FILE)

INFO:absl:Load dataset info from /home/jupyter/tensorflow_datasets/glue/mnli/1.0.0
INFO:absl:Reusing dataset glue (/home/jupyter/tensorflow_datasets/glue/mnli/1.0.0)
INFO:absl:Constructing tf.data.Dataset glue for split None, from /home/jupyter/tensorflow_datasets/glue/mnli/1.0.0


In [23]:
! gsutil ls {OUTPUT_LOCATION}

gs://jk-rs-notebook-test/datasets/MNLI/
gs://jk-rs-notebook-test/datasets/MNLI/mnli_train.tf_record
gs://jk-rs-notebook-test/datasets/MNLI/mnli_valid.tf_record


### Build a training container

There are two ways you can train a custom model using a container image:

- **Use a Google Cloud prebuilt container**. If you use a prebuilt container, you will additionally specify a Python package to install into the container image. 

- **Use your own custom container image**. If you use your own container, the container needs to contain your code for training a custom model.

In this example, you use a custom container image.

#### Create a training script

Your training script is based on [the common training driver](https://github.com/tensorflow/models/blob/master/official/nlp/docs/train.md) from TensorFlow NLP Modelling Toolkit. The base driver has been adapted to work seamlessly on a distributed compute environment provisioned when running a Vertex training job. 

In [ ]:
! rm -rf training_image
! mkdir training_image
! mkdir training_image/trainer

In [ ]:
%%writefile training_image/trainer/train.py

# Copyright 2021 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""TFM common training driver."""

import os
import json

from absl import app
from absl import flags
from absl import logging
import gin


from official.common import distribute_utils
# pylint: disable=unused-import
from official.common import registry_imports
# pylint: enable=unused-import
from official.common import flags as tfm_flags
from official.core import task_factory
from official.core import train_lib
from official.core import train_utils
from official.modeling import performance

from tensorflow.dtypes import float16, bfloat16, float32

FLAGS = flags.FLAGS

def _get_model_dir(model_dir):
  """Adjusts model dir for multi-worker training.
  
     Checkpointing and Saving need to happen on each worker and they need to write 
     to different paths as they would override each others. This utility function
     adjusts the base model dir passed as a flag using Vertex AI cluster topology
  """
  
  def _is_chief(task_type, task_id):
    return ((task_type == 'chief' and task_id == 0) or task_type is None)
  
  tf_config = os.getenv('TF_CONFIG')
  if tf_config:
    tf_config = json.loads(tf_config)
   
    if not _is_chief(tf_config['task']['type'], tf_config['task']['index']):
      model_dir = os.path.join(model_dir, 'worker-{}').format(tf_config['task']['index'])
  
  logging.info('Setting model_dir to: %s', model_dir)
  
  return model_dir

def main(_):
  
  model_dir = _get_model_dir(FLAGS.model_dir)

  gin.parse_config_files_and_bindings(FLAGS.gin_file, FLAGS.gin_params)
  params = train_utils.parse_configuration(FLAGS)
  
  if 'train' in FLAGS.mode:
    # Pure eval modes do not output yaml files. Otherwise continuous eval job
    # may race against the train job for writing the same file.
    train_utils.serialize_config(params, model_dir)

  # Sets mixed_precision policy. Using 'mixed_float16' or 'mixed_bfloat16'
  # can have significant impact on model speeds by utilizing float16 in case of
  # GPUs, and bfloat16 in the case of TPUs. loss_scale takes effect only when
  # dtype is float16
  if params.runtime.mixed_precision_dtype:
    # A mitigation for an apparent bug in passing mixed precision
    if params.runtime.mixed_precision_dtype == 'mixed_float16':
      precision_dtype = float16
    elif params.runtime_mixed_precision_dtype == 'mixed_bfloat16':
      precision_dtype = bfloat16
    else:
      precision_dtype = float32 
    performance.set_mixed_precision_policy(precision_dtype)
  distribution_strategy = distribute_utils.get_distribution_strategy(
      distribution_strategy=params.runtime.distribution_strategy,
      all_reduce_alg=params.runtime.all_reduce_alg,
      num_gpus=params.runtime.num_gpus,
      tpu_address=params.runtime.tpu,
      **params.runtime.model_parallelism())
  with distribution_strategy.scope():
    task = task_factory.get_task(params.task, logging_dir=model_dir)

  train_lib.run_experiment(
      distribution_strategy=distribution_strategy,
      task=task,
      mode=FLAGS.mode,
      params=params,
      model_dir=model_dir)

  train_utils.save_gin_config(FLAGS.mode, model_dir)

if __name__ == '__main__':
  tfm_flags.define_flags()
  app.run(main)


#### Create a base configuration for the MNLI fine tuning task

The training script supports overriding the default configuration of a TensorFlow Modelling Toolkit task using either YAML configuration files or command line parameters. The below YAML file sets the base configuration for the MNLI fine tuning task used in this example. 

In [ ]:
%%writefile training_image/trainer/glue_mnli_matched.yaml

task:
  hub_module_url: 'https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/4'
  model:
    num_classes: 3
  init_checkpoint: ''
  metric_type: 'accuracy'
  train_data:
    drop_remainder: true
    global_batch_size: 32
    input_path: ''
    is_training: true
    seq_length: 128
    label_type: 'int'
  validation_data:
    drop_remainder: false
    global_batch_size: 32
    input_path: ''
    is_training: false
    seq_length: 128
    label_type: 'int'
trainer:
  checkpoint_interval: 3000
  optimizer_config:
    learning_rate:
      polynomial:
        # 100% of train_steps.
        decay_steps: 36813
        end_learning_rate: 0.0
        initial_learning_rate: 3.0e-05
        power: 1.0
      type: polynomial
    optimizer:
      type: adamw
    warmup:
      polynomial:
        power: 1
        # ~10% of train_steps.
        warmup_steps: 3681
      type: polynomial
  steps_per_loop: 1000
  summary_interval: 1000
  # Training data size 392,702 examples, 3 epochs.
  train_steps: 36813
  validation_interval: 6135
  # Eval data size = 9815 examples.
  validation_steps: 307
  best_checkpoint_export_subdir: 'best_ckpt'
  best_checkpoint_eval_metric: 'cls_accuracy'
  best_checkpoint_metric_comp: 'higher'

#### Create the Dockerfile

The custom training container image packages TensorFlow NLP Modelling Toolkit with the training script and the default configuration file created in the previous steps. It also install the Reduction Server NCCL plugin.

In [ ]:
%%writefile training_image/Dockerfile

FROM gcr.io/deeplearning-platform-release/tf2-gpu.2-5

RUN apt remove -y google-fast-socket \
&&  echo "deb https://packages.cloud.google.com/apt google-fast-socket main" | tee /etc/apt/sources.list.d/google-fast-socket.list \
&&  curl -s -L https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add - \
&&  apt update && apt install -y google-reduction-server

RUN pip install tf-models-official==2.5.0 tensorflow-text==2.5.0

WORKDIR /

COPY trainer /trainer

ENTRYPOINT ["python"]
CMD ["-c", "print('TF Model Garden')"]

#### Build the container image and push it to Container Registry

In [ ]:
TRAIN_IMAGE = f'gcr.io/{PROJECT_ID}/mnli_finetuning'

In [ ]:
! docker build -t {TRAIN_IMAGE} training_image

In [ ]:
! docker push {TRAIN_IMAGE}

In [ ]:
!gcloud builds submit --tag {TRAIN_IMAGE} training_image

### Prepare training data

You can now prepare training and validation datasets in the format required by TensorFlow NLP Modelling Toolkit. Since this is 

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

{TODO: Include commands to delete individual resources below}

In [ ]:

# Delete model resource
! gcloud ai models delete $MODEL_NAME --quiet

# Delete Cloud Storage objects that were created
! gsutil -m rm -r $JOB_DIR